In [1]:
import os, sys
PARENT_FOLDER = os.path.join(os.path.dirname(os.path.realpath("__file__")), "../")
sys.path.append(PARENT_FOLDER)

import pandas as pd
import lost.utils as u
from datetime import datetime
from lost.connection import con

In [2]:
start_date = pd.to_datetime('01/01/2018')
end_date = '2021-07-12'

In [3]:
query = """
    select product_merch_classification2, count (product_merch_classification2) as frequency 
    from chewybi.products 
    group by product_merch_classification2 
    order by frequency desc
"""
mc2_unique = pd.read_sql_query(query, con)
mc2_unique.drop(28,0,inplace=True)
mc2_unique['mc2'] = mc2_unique['product_merch_classification2'].str.replace('&','and')
mc2_unique['mc2'] = mc2_unique['mc2'].str.replace(',','')

In [ ]:
mc2_unique.to_csv('../data/product_snowflake.csv')

In [ ]:
for i in range(mc2_unique.shape[0]):
    try:
        print('Started {}'.format(mc2_unique.loc[i,'product_merch_classification2']))
        args = {
        "start_date": start_date,
        "end_date": end_date,
        "class": mc2_unique.loc[i,'product_merch_classification2'],
        "file": mc2_unique.loc[i,'mc2'],
        }

        observation_period = (
        datetime.combine(start_date, datetime.min.time()), 
        datetime.combine(end_date, datetime.min.time())
        )

        FILE_PATH = os.path.join(PARENT_FOLDER, "data/V_{file}.pkl".format(**args))
        QUERY_PATH =  os.path.join(PARENT_FOLDER, "sql/lost_mc2-v2.sql")

        if os.path.exists(FILE_PATH): 
            continue
            df2 = pd.read_pickle(FILE_PATH)
        else:
            with open(QUERY_PATH, "r") as f:
                script = f.read().format(**args)
            df2 = u.snowflake_execute(script)
            df2.to_pickle(FILE_PATH)
    except:
        continue
        

In [ ]:
df = pd.DataFrame()
files = glob.glob("../data/V_*.pkl")
for f in files:
    data = pd.read_pickle(f)
    data['mc2'] = f.replace('../data/V_', '').replace('.pkl', '')
    df = pd.concat([df,data])

In [ ]:
df.to_pickle("../data/V_all.pkl")

In [ ]:
args = {
        "start_date": start_date,
        "end_date": end_date,
        }

query = """
        with base as (

            select distinct
            i.inventory_snapshot_snapshot_dt::date as date,
            i.product_part_number as item,
            p.product_merch_classification2 as mc2,
            sum(i.inventory_snapshot_sellable_quantity) as total_quantity,
            sum(i.inventory_snapshot_item_location_avg_daily_forecast_quantity) as forecast
            from chewybi.inventory_snapshot as i
            inner join chewybi.products as p
            using (product_part_number)
            where inventory_snapshot_snapshot_dt between '{start_date}' and '{end_date}'
            and item_location_product_published_flag
            and not item_location_product_discontinued_flag
            group by 1,3,2

        )

        select 
            date_trunc('week',b.date::date), -- get week interval 
            b.mc2,
            sum(case when total_quantity > 0 then 1 else 0 end) num_instock,
            sum(case when total_quantity > 0 then 0 else 1 end) as oos,
            count(item) as count,
            sum(case when total_quantity > 0 then 0 else 1 end)/count(item) as oos_rate,
            sum(case when total_quantity < 1 then b.forecast else 0 end) / sum(forecast) as weighted_oos_rate
        from base as b
        group by 1,2
        """

query2 = '''
   --- the query below can be used to look at the number of units ordered over a given period of time.  

        select date_trunc('week',olm.order_line_released_dttm::date)
        ,p.product_merch_classification2
        ,sum(olm.order_line_quantity) as ordered_qty
        ,sum(olm.order_line_total_price) as total_price
        ,sum(case when o.order_auto_reorder_flag is true then olm.order_line_quantity else 0 end) as autoship_qty -- autoships are the orders which are on a subscription. you can play around with this to see which
                                                                                                                -- types of items are more subscription havy than other products
        from chewybi.order_line_measures olm -- this table has details about the units purchased on each order. each order has an order id and order line id. each order line id
                                            -- is associated with one product, so we can the qty ordered for a given level of aggregation of products
        join chewybi.products p using(product_key) --- this table has product information such as product classification, brand, etc. t is keyed at the product level
        join chewybi.orders o using(order_key) -- this has order information, it is keyed at the order level
        where 1=1 
        and olm.order_line_released_dttm::date between date_trunc('week',TO_DATE({end_date}, 'YYYY-MM-DD')) - interval '4 years' and date_trunc('week',TO_DATE({end_date}, 'YYYY-MM-DD')) - interval '1 weeks' -- this is when the order was released to the warehouse to begin processing
        and o.order_status not in ('X','J') -- these are canceled orders, we always want to remove them
        group by 1,2
        ;
'''
dfo_all = pd.read_sql_query(query.format(**args), con)
oredered_all = pd.read_sql_query(query2.format(**args), con)

In [ ]:
dfo_all.to_csv('../data/dfo_all.csv')

In [ ]:
oredered_all.to_csv('../data/oredered_all.csv')